In [21]:
import cv2
import numpy as np

# Load original image
# def()
img = cv2.imread("iris_db/IITD_v1/orig/1_L/1_L_1.bmp")   # shape (H, W, 3)

# Load binary mask (white=keep, black=remove)
mask = cv2.imread("iris_db/IITD_v1/mask_irisseg/1_L/1_L_1.tiff", cv2.IMREAD_GRAYSCALE)  # shape (H, W)

# Ensure mask is 0 or 1
mask = (mask > 127).astype(np.uint8)

# Expand mask to 3 channels
mask_3ch = cv2.merge([mask, mask, mask])  # shape (H, W, 3)

# Apply mask to all 3 channels
segmented = img * mask_3ch

# Save result
cv2.imwrite("segmented_rgb.png", segmented)


True

Applied segmenetaton on dataset

In [ ]:
import os
import cv2
import numpy as np

def apply_mask(img_path, mask_path, save_path):
    # Read original image
    img = cv2.imread(img_path)

    # Read mask as grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Convert mask to binary (0 or 1)
    mask = (mask > 127).astype(np.uint8)

    # Expand to 3 channels
    mask_3ch = cv2.merge([mask, mask, mask])

    # Apply mask
    segmented = img * mask_3ch

    # Ensure output folder exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save result (always PNG)
    cv2.imwrite(save_path, segmented)


# ----------------- MAIN -----------------
root = "iris_db/IITD_v1"
segmented_root = "segmented_iitd_bmp"

orig_dir = os.path.join(root, "orig")
mask_dir = os.path.join(root, "mask_irisseg")

# Traverse subfolders in sorted order
for subfolder in sorted(os.listdir(orig_dir)):
    orig_sub = os.path.join(orig_dir, subfolder)
    mask_sub = os.path.join(mask_dir, subfolder)
    result_sub = os.path.join(segmented_root, subfolder)

    if not os.path.isdir(orig_sub):
        continue

    # Sorted files in each subfolder
    for filename in sorted(os.listdir(orig_sub)):
        if filename.lower().endswith(".bmp"):
            img_path = os.path.join(orig_sub, filename)

            # assume same stem but mask in .tiff
            stem = os.path.splitext(filename)[0]
            mask_path = os.path.join(mask_sub, stem + ".tiff")

            # force .png for saving
            save_path = os.path.join(result_sub, stem + ".bmp")

            if os.path.exists(mask_path):
                apply_mask(img_path, mask_path, save_path)
                print(f"Saved: {save_path}")
            else:
                print(f"Mask not found: {mask_path}")


In [ ]:
import os
import cv2
import numpy as np

def apply_mask(img_path, mask_path, save_path):
    """
    Apply a binary mask to an image:
    - Keeps original pixels where mask == 1
    - Background (mask == 0) becomes black
    """

    # Read original image in color
    img = cv2.imread(img_path)   # shape (H, W, 3), dtype uint8

    # Read mask as grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # shape (H, W)

    # Convert mask to binary (values 0 or 1)
    mask = (mask > 127).astype(np.uint8)

    # Expand to 3 channels so it matches the original image shape
    mask_3ch = cv2.merge([mask, mask, mask])  # shape (H, W, 3)

    # Apply mask: keep original where mask == 1, else black
    segmented = img * mask_3ch

    # Ensure output folder exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save result (you can use .png or .bmp)
    cv2.imwrite(save_path, segmented)




In [ ]:
from tqdm import tqdm_notebook as tqdm
import os
import cv2
import numpy as np
from pathlib import Path
# from matplotlib import pyplot as plt

# ----------------- MAIN -----------------
root = "iris_db/IITD_v1"
segmented_root = "overlay_iitd_bmp"

orig_dir = os.path.join(root, "orig")
mask_dir = os.path.join(root, "mask_irisseg")

def save_overlay_image(original_image, iris_mask, out_file):
    # Load the original image and the iris mask
    original_image = cv2.imread(original_image)
    iris_mask = cv2.imread(iris_mask, cv2.IMREAD_UNCHANGED)
    iris_mask_colored = cv2.cvtColor(iris_mask, cv2.COLOR_GRAY2BGR)
    # set the color of the mask to red
    iris_mask_colored[np.where((iris_mask_colored != [0, 0, 0]).all(axis=2))] = [200, 0, 0]
    overlay = cv2.addWeighted(original_image, 0.9, iris_mask_colored, 0.8, 1)
    # show overlay image without axis
    # plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB), cmap='gray', vmin=0, vmax=255, aspect='auto', interpolation='nearest')
    Path(out_file).parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(out_file, overlay)

# Traverse subfolders in sorted order
for subfolder in tqdm(sorted(os.listdir(orig_dir))):
    orig_sub = os.path.join(orig_dir, subfolder)
    mask_sub = os.path.join(mask_dir, subfolder)
    result_sub = os.path.join(segmented_root, subfolder)

    if not os.path.isdir(orig_sub):
        continue

    # Sorted files in each subfolder
    for filename in sorted(os.listdir(orig_sub)):
        if filename.lower().endswith(".bmp"):
            img_path = os.path.join(orig_sub, filename)

            # assume same stem but mask in .tiff
            stem = os.path.splitext(filename)[0]
            mask_path = os.path.join(mask_sub, stem + ".tiff")

            # force .png for saving
            save_path = os.path.join(result_sub, stem + ".png")

            if os.path.exists(mask_path):
                save_overlay_image(img_path, mask_path, save_path)
                print(f"Saved: {save_path}")
            else:
                print(f"Mask not found: {mask_path}")



  3%|▎         | 13/435 [00:00<00:03, 124.28it/s]

Saved: overlay_iitd_bmp/100_L/100_L_1.bmp
Saved: overlay_iitd_bmp/100_L/100_L_2.bmp
Saved: overlay_iitd_bmp/100_L/100_L_3.bmp
Saved: overlay_iitd_bmp/100_L/100_L_4.bmp
Saved: overlay_iitd_bmp/100_L/100_L_5.bmp
Saved: overlay_iitd_bmp/100_R/100_R_1.bmp
Saved: overlay_iitd_bmp/100_R/100_R_2.bmp
Saved: overlay_iitd_bmp/100_R/100_R_3.bmp
Saved: overlay_iitd_bmp/100_R/100_R_4.bmp
Saved: overlay_iitd_bmp/100_R/100_R_5.bmp
Saved: overlay_iitd_bmp/101_L/101_L_1.bmp
Saved: overlay_iitd_bmp/101_L/101_L_2.bmp
Saved: overlay_iitd_bmp/101_L/101_L_3.bmp
Saved: overlay_iitd_bmp/101_L/101_L_4.bmp
Saved: overlay_iitd_bmp/101_L/101_L_5.bmp
Saved: overlay_iitd_bmp/101_R/101_R_1.bmp
Saved: overlay_iitd_bmp/101_R/101_R_2.bmp
Saved: overlay_iitd_bmp/101_R/101_R_3.bmp
Saved: overlay_iitd_bmp/101_R/101_R_4.bmp
Saved: overlay_iitd_bmp/101_R/101_R_5.bmp
Saved: overlay_iitd_bmp/102_L/102_L_1.bmp
Saved: overlay_iitd_bmp/102_L/102_L_2.bmp
Saved: overlay_iitd_bmp/102_L/102_L_3.bmp
Saved: overlay_iitd_bmp/102_L/102_

  9%|▉         | 39/435 [00:00<00:03, 125.24it/s]

Saved: overlay_iitd_bmp/112_L/112_L_1.bmp
Saved: overlay_iitd_bmp/112_L/112_L_2.bmp
Saved: overlay_iitd_bmp/112_L/112_L_3.bmp
Saved: overlay_iitd_bmp/112_L/112_L_4.bmp
Saved: overlay_iitd_bmp/112_L/112_L_5.bmp
Saved: overlay_iitd_bmp/112_R/112_R_1.bmp
Saved: overlay_iitd_bmp/112_R/112_R_2.bmp
Saved: overlay_iitd_bmp/112_R/112_R_3.bmp
Saved: overlay_iitd_bmp/112_R/112_R_4.bmp
Saved: overlay_iitd_bmp/112_R/112_R_5.bmp
Saved: overlay_iitd_bmp/113_L/113_L_1.bmp
Saved: overlay_iitd_bmp/113_L/113_L_2.bmp
Saved: overlay_iitd_bmp/113_L/113_L_3.bmp
Saved: overlay_iitd_bmp/113_L/113_L_4.bmp
Saved: overlay_iitd_bmp/113_L/113_L_5.bmp
Saved: overlay_iitd_bmp/113_R/113_R_1.bmp
Saved: overlay_iitd_bmp/113_R/113_R_2.bmp
Saved: overlay_iitd_bmp/113_R/113_R_3.bmp
Saved: overlay_iitd_bmp/113_R/113_R_4.bmp
Saved: overlay_iitd_bmp/113_R/113_R_5.bmp
Saved: overlay_iitd_bmp/114_L/114_L_1.bmp
Saved: overlay_iitd_bmp/114_L/114_L_2.bmp
Saved: overlay_iitd_bmp/114_L/114_L_3.bmp
Saved: overlay_iitd_bmp/114_L/114_

 15%|█▍        | 65/435 [00:00<00:02, 123.86it/s]

...

Saved: overlay_iitd_bmp/64_L/64_L_3.bmp
Saved: overlay_iitd_bmp/64_L/64_L_4.bmp
Saved: overlay_iitd_bmp/64_L/64_L_5.bmp
Saved: overlay_iitd_bmp/64_R/64_R_1.bmp
Saved: overlay_iitd_bmp/64_R/64_R_2.bmp
Saved: overlay_iitd_bmp/64_R/64_R_3.bmp
Saved: overlay_iitd_bmp/64_R/64_R_4.bmp
Saved: overlay_iitd_bmp/64_R/64_R_5.bmp
Saved: overlay_iitd_bmp/65_L/65_L_1.bmp
Saved: overlay_iitd_bmp/65_L/65_L_2.bmp
Saved: overlay_iitd_bmp/65_L/65_L_3.bmp
Saved: overlay_iitd_bmp/65_L/65_L_4.bmp
Saved: overlay_iitd_bmp/65_L/65_L_5.bmp
Saved: overlay_iitd_bmp/65_L/65_L_6.bmp
Saved: overlay_iitd_bmp/65_R/65_R_2.bmp
Saved: overlay_iitd_bmp/65_R/65_R_3.bmp
Saved: overlay_iitd_bmp/65_R/65_R_4.bmp
Saved: overlay_iitd_bmp/65_R/65_R_5.bmp
Saved: overlay_iitd_bmp/66_L/66_L_1.bmp
Saved: overlay_iitd_bmp/66_L/66_L_2.bmp
Saved: overlay_iitd_bmp/66_L/66_L_3.bmp
Saved: overlay_iitd_bmp/66_L/66_L_4.bmp
Saved: overlay_iitd_bmp/66_L/66_L_5.bmp
Saved: overlay_iitd_bmp/66_R/66_R_1.bmp
Saved: overlay_iitd_bmp/66_R/66_R_2.bmp


 92%|█████████▏| 402/435 [00:03<00:00, 128.03it/s]

Saved: overlay_iitd_bmp/77_L/77_L_1.bmp
Saved: overlay_iitd_bmp/77_L/77_L_2.bmp
Saved: overlay_iitd_bmp/77_L/77_L_3.bmp
Saved: overlay_iitd_bmp/77_L/77_L_4.bmp
Saved: overlay_iitd_bmp/77_L/77_L_5.bmp
Saved: overlay_iitd_bmp/77_R/77_R_1.bmp
Saved: overlay_iitd_bmp/77_R/77_R_2.bmp
Saved: overlay_iitd_bmp/77_R/77_R_3.bmp
Saved: overlay_iitd_bmp/77_R/77_R_4.bmp
Saved: overlay_iitd_bmp/77_R/77_R_5.bmp
Saved: overlay_iitd_bmp/78_L/78_L_1.bmp
Saved: overlay_iitd_bmp/78_L/78_L_2.bmp
Saved: overlay_iitd_bmp/78_L/78_L_3.bmp
Saved: overlay_iitd_bmp/78_L/78_L_4.bmp
Saved: overlay_iitd_bmp/78_L/78_L_5.bmp
Saved: overlay_iitd_bmp/78_R/78_R_1.bmp
Saved: overlay_iitd_bmp/78_R/78_R_2.bmp
Saved: overlay_iitd_bmp/78_R/78_R_3.bmp
Saved: overlay_iitd_bmp/78_R/78_R_4.bmp
Saved: overlay_iitd_bmp/78_R/78_R_5.bmp
Saved: overlay_iitd_bmp/79_L/79_L_1.bmp
Saved: overlay_iitd_bmp/79_L/79_L_2.bmp
Saved: overlay_iitd_bmp/79_L/79_L_3.bmp
Saved: overlay_iitd_bmp/79_L/79_L_4.bmp
Saved: overlay_iitd_bmp/79_L/79_L_5.bmp


100%|██████████| 435/435 [00:03<00:00, 127.31it/s]

Saved: overlay_iitd_bmp/89_R/89_R_2.bmp
Saved: overlay_iitd_bmp/89_R/89_R_3.bmp
Saved: overlay_iitd_bmp/89_R/89_R_4.bmp
Saved: overlay_iitd_bmp/89_R/89_R_5.bmp
Saved: overlay_iitd_bmp/8_L/8_L_1.bmp
Saved: overlay_iitd_bmp/8_L/8_L_10.bmp
Saved: overlay_iitd_bmp/8_L/8_L_2.bmp
Saved: overlay_iitd_bmp/8_L/8_L_3.bmp
Saved: overlay_iitd_bmp/8_L/8_L_4.bmp
Saved: overlay_iitd_bmp/8_L/8_L_5.bmp
Saved: overlay_iitd_bmp/8_L/8_L_6.bmp
Saved: overlay_iitd_bmp/8_L/8_L_7.bmp
Saved: overlay_iitd_bmp/8_L/8_L_8.bmp
Saved: overlay_iitd_bmp/8_L/8_L_9.bmp
Saved: overlay_iitd_bmp/90_L/90_L_1.bmp
Saved: overlay_iitd_bmp/90_L/90_L_2.bmp
Saved: overlay_iitd_bmp/90_L/90_L_3.bmp
Saved: overlay_iitd_bmp/90_L/90_L_4.bmp
Saved: overlay_iitd_bmp/90_L/90_L_5.bmp
Saved: overlay_iitd_bmp/90_R/90_R_1.bmp
Saved: overlay_iitd_bmp/90_R/90_R_2.bmp
Saved: overlay_iitd_bmp/90_R/90_R_3.bmp
Saved: overlay_iitd_bmp/90_R/90_R_4.bmp
Saved: overlay_iitd_bmp/90_R/90_R_5.bmp
Saved: overlay_iitd_bmp/91_L/91_L_1.bmp
Saved: overlay_iitd